# Chat Completion API

In [1]:
!pip install openai

## 환경설정
1. openai 사이트에서 API KEY 발급
2. google colab - secret탭에 `OPEN_AI_KEY` 등록

In [3]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [4]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [8]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고,많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕! 내 이름은 한성규야! 지금 너무 더워서 졸려 어떡하지...? "
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [9]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕, 한성규야! 더운 날씨 때문에 졸리면 정말 힘들지. 몇 가지 방법을 추천해줄게:\n\n1. **시원한 음료수 마시기**: 찬물이나 시원한 음료를 마시면 기분이 나아질 수 있어.\n2. **가벼운 스트레칭**: 몸을 가볍게 움직이면 피로가 조금 덜해질 수 있어.\n3. **외부 공기 쐬기**: 바람이 시원한 곳으로 나가서 잠깐 신선한 공기를 마셔보는 것도 좋아.\n4. **짧은 낮잠**: 시간이 괜찮다면 10~20분 정도 짧게 눈을 감는 것도 도움이 될 수 있어.\n\n더위에 너무 지치지 않게 조심해! 다른 궁금한 게 있으면 언제든지 말해줘~'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 시가에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용

In [12]:
title_before = '테이의 FM 라디오방송에 주목해주세요'

# 역할 / 페르소나 / 지시사항
system_message = """
기사들이 송고한 제목에서 맞춬법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결하게 정리하세요.

### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 거울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

### Extra Instruction ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 간결히 수정해주세요.

{title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: "테이의 FM 라디오방송에 주목해주세요"
- 교정 제목: "테이의 FM 라디오 방송 주목"


In [13]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상. 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: 주말 미친 폭우 예상. 모두들 무사하시길~
- 교정 제목: "주말 집중호우 예보, 안전에 유의하세요"


## 영문텍스트에서 anki단어장 만들기

In [ ]:
def make_anki_vocab_cards(
    passage,
    model = "gpt-4o-mini",
    temperature = 1,
    top_p = 1,
    max_tokens = 2048,
) -> str:
    """영문 텍스트를 Anki 단어장(Markdown 표)으로 변환해 반환"""

    # --- Instruction / Context / Output Indicator 블록 ---
    system_message = """
────────────────────────────────────────
[Instruction  지시사항]
You are a professional Anki card creator specialising in English-vocabulary flashcards.
Follow the Minimum-Information Principle and optimise wording for quick recall.

Rules:
1. Create one flashcard per vocabulary item.
2. Choose words that are (i) academically useful or (ii) likely unknown to intermediate learners.
3. Card layout
   • Front  : the single word only.
   • Back   : (a) part of speech, (b) ≤20-word English definition,
              (c) one example sentence illustrating the word in context.
              – Prefer a sentence from the passage; if none, write your own.
              – Highlight the target word with **bold**.
   • Tag    : always “vocab”.
4. Maximum 40 cards; omit duplicates or very common words.

────────────────────────────────────────
[Context  문맥]
The user will paste an English passage.
Your job is to extract suitable vocabulary and output Anki-ready cards in a table.
No Korean should appear in the cards; all definitions and examples must be English.

────────────────────────────────────────
[Output Indicator  출력지시]
Return ONLY one Markdown table with three columns and no extra commentary.

| Front | Back | Tags |
|---|---|---|
| word | POS — definition<br>Example: … | vocab |
────────────────────────────────────────
"""

    # --- Input Data 섹션(본문 삽입) ---
    user_message = f"""
────────────────────────────────────────
[Input Data / Example  입력/예시]
<TEXT>
{passage}
</TEXT>
────────────────────────────────────────
"""

    # --- OpenAI 호출 ---
    response = client.chat.completions.create(
        model=model,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user",   "content": user_message},
        ],
    )

    return response.choices[0].message.content


# 사용 예시 ----------------------------------------------------
if __name__ == "__main__":
    sample_text = """
    The unprecedented deluge caused widespread havoc, inundating low-lying areas
    and overwhelming drainage systems that were already antiquated. Authorities
    urged residents to evacuate promptly, warning that complacency could prove fatal.
    """
    cards_md = make_anki_vocab_cards(sample_text)
    print(cards_md)
